%%


 <br>
import all packages needed for this bit<br>
<br>
%%

In [ ]:
import pandas as pd
from sodapy import Socrata
import datetime
import matplotlib.pyplot as plt
import numpy as np

%% 


<br>
set up connection to the data source<br>
<br>
%%

In [ ]:
domain = 'data.sfgov.org'
app_token = 'TQqk1kfcIaDLukBlSTtay4NNz'

In [ ]:
client = Socrata("data.sfgov.org", app_token, timeout = 30)

dictionaries by sodapy.

In [ ]:
dataset = 'nuek-vuh3'
metadata = client.get_metadata(dataset)
columns = metadata.get('columns')
fieldNames = [col['fieldName'] for col in columns]

%% 


define function for extracting relevant fields from data source<br>
<br>
%% 

In [ ]:
def getAll(domain, app_token, dataset,fieldList,limit):
    client = Socrata(domain, app_token, timeout = 30)
    results = list()
    for i in range(limit):        
        incoming = client.get(dataset, select = fieldList)
        results.append(incoming)
       
        if len(incoming)<1000:
            break
        pctComplete = 100* (len(results)/limit)
        if pctComplete % 10==0:
            print('percent complete :',pctComplete,  ' %')
            
    return results

In [ ]:
fieldList = 'received_dttm, dispatch_dttm, response_dttm, call_Type, fire_prevention_district, neighborhoods_analysis_boundaries, number_of_alarms, original_priority, priority, unit_type'

%% 


<br>
define function for converting JSON to a pandas dataframe<br>
<br>
%% 

In [ ]:
def jsonToDF(json):
    results_df = pd.DataFrame()
    for i in range(len(json)):
        results_df= results_df.append(pd.DataFrame.from_records(json[i]))
    return results_df
            

In [ ]:
results = getAll(domain,app_token,dataset,fieldList,limit=500)
results_df = jsonToDF(results)        

%% 


set up data schema (data types of columns)<br>
<br>
%% 

In [ ]:
dataTypeDict = {'received_dttm':'datetime64',
                'dispatch_dttm':'datetime64',
                'response_dttm':'datetime64',
                'call_Type':'str',
                'fire_prevention_district':'str',
                'neighborhoods_analysis_boundaries':'str',
                'number_of_alarms':'int',
                'original_priority':'str',
                'priority':'str',
                'unit_type':'str'}    
results_df = results_df.astype(dataTypeDict)

%% 


do some preliminary math on dataset<br>
<br>
%% 

In [ ]:
results_df['response duration'] = results_df['response_dttm'] - results_df['received_dttm']
results_df['response duration'] = results_df['response duration']/np.timedelta64(1,'s')/60

In [ ]:
results_df['travel time'] = results_df['response_dttm'] - results_df['dispatch_dttm']
results_df['travel time'] = results_df['travel time']/np.timedelta64(1,'s')/60

In [ ]:
results_df.dtypes
        